# Recoinize

[https://github.com/nicknochnack/FaceRecognition](https://github.com/nicknochnack/FaceRecognition/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network%20-%20Final.ipynb)

# Setup

## Pull code from GitHub

In [ ]:
from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
git_update("live-edit", force=False, show_output=False)
# install_requirements()

In [6]:
print_gpu_name()

'NVIDIA RTX A4000\n'

## Import dependencies

In [2]:
testing = True

In [3]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
if not testing:
    import tensorflow as tf
    from tensorflow import keras
    from keras.layers import Dense, Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
    from keras.models import Sequential, load_model, Model
    from datasets import load_dataset

    from src.models import make_embedding, make_siamese_model
    from src.helpers.setup import set_gpus_growth
    from src.dataset import preprocess, create_pairs

    set_gpus_growth()
else:
    print("Testing mode, skipping tensorflow imports")
    from src.helpers.load_data import load_data

Testing mode, skipping tensorflow imports


# Get and preprocess data

In [4]:
# Load the dataset from HuggingFace
if not testing:
    dataset = load_dataset('photonsquid/coins-euro')
    dataset = dataset.with_format('tf')
else:
    # load data from local files
    dataset = load_data("data/tests")


coucou


In [5]:
print(dataset)

{'train': [{'image': array([[[ 86,  87,  87],
        [ 84, 107, 107],
        [ 78, 105, 108],
        ...,
        [ 47,  65,  61],
        [ 48,  67,  60],
        [ 35,  51,  47]],

       [[ 84, 105,  97],
        [ 82, 112, 103],
        [ 75,  97,  88],
        ...,
        [ 61,  91,  86],
        [ 67,  93,  86],
        [ 43,  63,  57]],

       [[ 80, 108, 100],
        [ 84, 109,  98],
        [ 84, 111, 102],
        ...,
        [ 64,  87,  78],
        [ 62,  86,  82],
        [ 45,  71,  67]],

       ...,

       [[ 67,  92,  87],
        [ 73, 101,  93],
        [ 72, 101,  95],
        ...,
        [ 67,  95,  89],
        [ 64,  93,  89],
        [ 65,  99,  95]],

       [[ 69,  96,  87],
        [ 69,  99,  92],
        [ 67, 100,  95],
        ...,
        [ 66,  95,  92],
        [ 63,  91,  89],
        [ 74, 102,  94]],

       [[ 70,  99,  89],
        [ 77, 100,  95],
        [ 71,  97,  92],
        ...,
        [ 59,  92,  90],
        [ 70, 105,  97],
   

In [10]:
# Split the dataset into train and test
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Preprocess the data
train_dataset['image'].map(preprocess)
test_dataset['image'].map(preprocess)

In [ ]:
# create the pairs and labels for the training dataset
train_dataset.map(create_pairs)

# create the pairs and labels for the testing dataset
test_pairs, test_labels = create_pairs(test_dataset['image'], test_dataset['label'])

In [ ]:
# prepare the dataset for training
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# prepare the dataset for testing
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)